# Import libraries

In [23]:
import numpy as np
import random
import operator
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats

# Parameters

In [24]:
Distance = [[0, 32, 39, 42, 29, 35],
           [32, 0, 36, 27, 41, 25], 
           [39, 36, 0, 28, 33, 40], 
           [42, 27, 28, 0, 27, 38],
           [29, 41, 33, 27, 0, 26],
           [35, 25, 40, 38, 26, 0]]  #distance matrix between the cities

n = 6            #number of cities visited
p = 8            #number of candidate solutions in the population
par = 6          #number of parents used in the generation step
k = 3            #tournament size

# Initial population

In [25]:
initial = [1, 2, 3, 4, 5, 6]  #these are all the possible cities that can be visited (the order must be randomised)
initial_population = []

for i in range(p):
    solution = initial.copy()
    random.shuffle(solution)   #the order of the possible cities is randomised to generate an initial population
    initial_population.append(solution)

print(initial_population)

[[4, 3, 6, 1, 5, 2], [1, 6, 3, 5, 2, 4], [1, 6, 3, 2, 4, 5], [6, 2, 1, 3, 4, 5], [6, 1, 2, 4, 5, 3], [1, 5, 4, 6, 3, 2], [2, 1, 4, 5, 6, 3], [3, 1, 6, 4, 2, 5]]


# Model

## Fitness (For the TSP problem this is minimising the distance)

Objective function is not the fitness function - it is a minimisation problem

In [26]:
def distance(population):    #The distance between the cities is the objective function
    
    for i in range(len(population)):
        
        distance = []

        for i in range(len(population)-1): 
            
            #The distance is the difference between points (it returns one less than the number of cities)

            a = population[i]        #The first city 
            b = population[i+1]      #The second city

            dist = Distance[a-1][b-1]   #The point on the distance matrix of the first and secod cities
            distance.append(dist)


        a = population[0]            #The first city in the whole solution
        b = population[n - 1]        #The last city in the whole solution

        dist = Distance[a-1][b-1]    #The distance between the first and last city (closing the loop)

        distance.append(dist)
        total_distance = sum(distance)   #The sum of distances between all of the cities
    
    return total_distance

In [27]:
print(initial_population[0])
distance(initial_population[0])

[4, 3, 6, 1, 5, 2]


200

In [28]:
def fitness(population):    #The fitness of the solution is not equal to the objective function of a minimisation problem
    
    fitness = []
    
    for i in range(len(population)):
        
        dist = distance(population[i])
        inverse = 1/dist                    
        
        #The TSP is a minimisation problem therefore the inverse needs to be used to get the highest fitness value for a minimum
        
        fitness.append(inverse*10000)  #The inverse is mulitplied by a chosen number to get a more usable value
        
    return fitness

In [29]:
#print(initial_population[0])
fitness(initial_population)

[50.0,
 45.87155963302752,
 51.54639175257732,
 56.497175141242934,
 51.54639175257732,
 49.504950495049506,
 49.26108374384236,
 46.948356807511736]

# Best initial conditions

In [30]:
best_route = initial_population[0]
best_distance = distance(best_route)

print(best_route)   
print(best_distance)

initial_distance = []

for i in range(p):
    dist = distance(initial_population[i])
    initial_distance.append(dist)
    
    if dist < best_distance:
        best_distance = dist
        best_route = initial_population[i]
        
print(best_route)   
print(best_distance)
print(initial_distance)

[4, 3, 6, 1, 5, 2]
200
[6, 2, 1, 3, 4, 5]
177
[200, 218, 194, 177, 194, 202, 203, 213]


# Probabilities based on fitness

In [31]:
def probability(population):   #The probability of a solution being selected depends on the relative fitness of the solution

    probability = []
    
    fit = fitness(population)
    
    total = sum(fit)                      #The total fitness of the population is calculated

    probability = np.array(fit) / total     
    
    #The probability is calculated as the fitness of the solution relative to the total fitness of the population
    
    return probability

In [32]:
probability(initial_population)

array([0.12463361, 0.11434276, 0.12848825, 0.14082893, 0.12848825,
       0.12339961, 0.12279173, 0.11702686])

# Tournament selection

In [33]:
def tournament(population):

    parents = []

    pop = [0, 1, 2, 3, 4, 5, 6, 7]  #These are the references of the solutions in the population

    for i in range(6):   #only 6 parents can be chosen based on the tournament size
        
        populate = []
        
        for i in range(len(pop)):
            
            index = pop[i]
            populate.append(population[index])
        
        prob = probability(populate)    #The probability of the remaining solutions are calculated as solutions are removed

        choice = np.random.choice(pop, 3, replace=False, p = prob)   
        
        #3 unique values are chosen from the possible solutions in the population based on probabilities

        tournament_dist = 600   #This is a random value used as a starting point

        for i in range(3):

            dist = distance(population[choice[i]])

            if dist < tournament_dist:    #This ensures that the smallest distance is the winning distance
                tournament_dist = dist
                tournament_choice = population[choice[i]]
                winner = choice[i]

        parents.append(population[winner])  #The smallest distance of the chosen 3 is the winner and becomes a parent

        pop.remove(winner)  
        
#The solution has been chosen as a parent and needs to be removed so that a new solution can be chosen as a different parent

    return parents    #a list of the 6 parents are chosen

In [34]:
print(initial_population)
par = tournament(initial_population)
print(par)

[[4, 3, 6, 1, 5, 2], [1, 6, 3, 5, 2, 4], [1, 6, 3, 2, 4, 5], [6, 2, 1, 3, 4, 5], [6, 1, 2, 4, 5, 3], [1, 5, 4, 6, 3, 2], [2, 1, 4, 5, 6, 3], [3, 1, 6, 4, 2, 5]]
[[6, 1, 2, 4, 5, 3], [1, 6, 3, 2, 4, 5], [6, 2, 1, 3, 4, 5], [4, 3, 6, 1, 5, 2], [2, 1, 4, 5, 6, 3], [1, 5, 4, 6, 3, 2]]


# Crossover operator (2-point crossover)

In [35]:
def breed(parents):

    geneA = int(random.random() * (len(parents[0])-1))  #the minus 1 ensures that there is at least one split in the gene
    geneB = int(random.random() * (len(parents[0])-1))  #This returns a random index value in the solution
    
    startGene = min(geneA, geneB)       #The staring point needs to be the smaller index value
    endGene = max(geneA, geneB)         #The ending point needs to be the larger index value
    
    par = [0, 1, 2, 3, 4, 5]
    children = []

    for i in range(3):
        child1 = [0, 0, 0, 0, 0, 0]
        child2 = [0, 0, 0, 0, 0, 0]
        childP12 = []
        childP22 = []

        parent = np.random.choice(par, 2, replace=False)   #randomly choose two parents to mate from the list of parents
        parent1 = parents[parent[0]]
        parent2 = parents[parent[1]]
        
        for i in range(startGene, endGene):  #The portion of parent 1 that will be in child 1 inbetween the splits
            child1[i] = parent1[i]
        for i in range(startGene, endGene):  #The portion of parent 2 that will be in child 2 inbetween the splits
            child2[i] = parent2[i] 
        for i in range(len(child1)):
            if child1[i] == 0:
                childP12 = [item for item in parent2 if item not in child1]#The genes from parent 2 that are not in child 1 yet
                child1[i] = childP12[0]
        for i in range(len(child2)):
            if child2[i] == 0:
                childP22 = [item for item in parent1 if item not in child2]#The genes from parent 1 that are not in child 2 yet
                child2[i] = childP22[0]
                
        par.remove(parent[0])         #remove the parents from the list after they have mated so that there is no repeat
        par.remove(parent[1])
        
        children.append(child1)
        children.append(child2)

    print([startGene, endGene])

    return children

In [36]:
#print(par[0], par[1])

print(breed(initial_population))

[4, 4]
[[1, 6, 3, 5, 2, 4], [1, 6, 3, 2, 4, 5], [6, 2, 1, 3, 4, 5], [4, 3, 6, 1, 5, 2], [1, 5, 4, 6, 3, 2], [6, 1, 2, 4, 5, 3]]


# Mutation

In [37]:
def mutation(child):
    
    selected = int(random.random() * (len(child)))  #A random child is selected from the list of children
    
    Swap1 = int(random.random() * (len(child[selected])))  #A gene is chosen at random to be swapped
    Swap2 = int(random.random() * (len(child[selected])))  #A gene is chosen at random to be swapped with
    
    swap = child[selected]
    
    Gene1 = swap[Swap1]
    Gene2 = swap[Swap2]
    
    swap[Swap1] = Gene2  #The two selected genes are swapped
    swap[Swap2] = Gene1
    
    print([Swap1, Swap2])
    return selected, swap  #The index of the selected child and the new child with the mutation is returned

In [38]:
print(initial_population)
mute = mutation(initial_population)
print(mute)

[[4, 3, 6, 1, 5, 2], [1, 6, 3, 5, 2, 4], [1, 6, 3, 2, 4, 5], [6, 2, 1, 3, 4, 5], [6, 1, 2, 4, 5, 3], [1, 5, 4, 6, 3, 2], [2, 1, 4, 5, 6, 3], [3, 1, 6, 4, 2, 5]]
[5, 3]
(3, [6, 2, 1, 5, 4, 3])


In [39]:
print(initial_population)
initial_population.remove(initial_population[mute[0]])
initial_population.append(mute[1])
print(initial_population)

[[4, 3, 6, 1, 5, 2], [1, 6, 3, 5, 2, 4], [1, 6, 3, 2, 4, 5], [6, 2, 1, 5, 4, 3], [6, 1, 2, 4, 5, 3], [1, 5, 4, 6, 3, 2], [2, 1, 4, 5, 6, 3], [3, 1, 6, 4, 2, 5]]
[[4, 3, 6, 1, 5, 2], [1, 6, 3, 5, 2, 4], [1, 6, 3, 2, 4, 5], [6, 1, 2, 4, 5, 3], [1, 5, 4, 6, 3, 2], [2, 1, 4, 5, 6, 3], [3, 1, 6, 4, 2, 5], [6, 2, 1, 5, 4, 3]]


# Replacement (Elitism)

In [40]:
def elitism(population):
    
    fit = fitness(population)           #The fitness of all the solution in the population is calculated
    
    new = []
    i = 0

    while len(new) <= 8:          #This is to ensure that the top 8 fittest solutions are returned

        max = np.max(fit)         #The maximum fitness in the population set is found
        conditon = (fit == max)
        result = np.where(conditon)[0]   #The index value of the maximum fitness is found

        if len(result) > 1:              #if there are multiple solutions with the same fitness

            for i in range(len(result)):
                
                if i == 0:
                    new.append(population[result[i]])     #add the solution to the new population
                    fit.remove(fit[result[i]])            #remove the solution from the list of options
                    population.remove(population[result[i]])
                
                else:
                    result = result - 1                
                    new.append(population[result[i]])  
                    fit.remove(fit[result[i]])
                    population.remove(population[result[i]])

        else:
            new.append(population[result[0]])             #add the solution to the new population
            fit.remove(fit[result[0]])                    #remove the solution from the list of option
            population.remove(population[result[0]])

    return new[:8]      #return the 8 solutions with the highest fitness values

In [41]:
population = initial_population + initial_population
elitism(population)

[[6, 2, 1, 5, 4, 3],
 [6, 2, 1, 5, 4, 3],
 [1, 6, 3, 2, 4, 5],
 [6, 1, 2, 4, 5, 3],
 [1, 6, 3, 2, 4, 5],
 [6, 1, 2, 4, 5, 3],
 [4, 3, 6, 1, 5, 2],
 [4, 3, 6, 1, 5, 2]]

# Best solution

In [42]:
initial_distance = []   #find the initial distances of the initial population

for i in range(p):
    dist = distance(initial_population[i])
    initial_distance.append(dist)
    
    if dist < best_distance:
        best_distance = dist                   #Find the shortest distance from the initial population
        best_route = initial_population[i]     #Find the route that corresponds to the shortest distance
        
print(best_route)   
print(best_distance)
print(initial_distance)

[6, 2, 1, 5, 4, 3]
177
[200, 218, 194, 194, 202, 203, 213, 181]


# Genetic Algorithm

In [43]:
generation = 0    #initialise the generation 
no_change = 0

population = initial_population  #initialise the population to the initial, random population

current_route = population[0]
current_distance = distance(current_route)

while no_change < 8:     #The algorithm will run until there is no imporvement in 8 generations
    
    print(generation)
    
    population_distance = []         #find the distances of the population
    
    print(population)    

    for i in range(8):
        dist = distance(population[i])
        population_distance.append(dist)
            
    fit = fitness(population)   #find the fitness of the population

    print(population_distance)
    print(fit)

    parents = tournament(population)

    print(parents)

    children = breed(parents)

    print(children)

    mut = mutation(children)         #mutate one of the children

    print(mut)

    children.remove(children[mut[0]])   #remove the original child and include the mutated child
    children.append(mut[1])

    population = children + population             #The total population is the old population plus the new children
    
    mean_fit_all = np.mean(fitness(population))     #find the mean fitness of the old population and children
    
    print(mean_fit_all)

    new_population = elitism(population)            #find the new population from elitism selection
    
    mean_fit_new = np.mean(fitness(new_population)) #find the mean fitness of the new population
    print(mean_fit_new)
    
    population = new_population

    population_distance = []

    for i in range(8):
        dist = distance(population[i])
        population_distance.append(dist)

        if dist < best_distance:                 #in the new population distances find if there is a smaller distance
            best_distance = dist
            best_route = population[i]
    
    print(population_distance)
    print(best_route)
    print(best_distance)

    if best_distance == current_distance:         #if there were no improving solutions in the neighborhood
        no_change = no_change + 1

    else:
        current_route = best_route                #else: continue to the next generation with the new population
        current_distance = best_distance
        no_change = 0
       
    generation = generation + 1


0
[[4, 3, 6, 1, 5, 2], [1, 6, 3, 5, 2, 4], [1, 6, 3, 2, 4, 5], [6, 1, 2, 4, 5, 3], [1, 5, 4, 6, 3, 2], [2, 1, 4, 5, 6, 3], [3, 1, 6, 4, 2, 5], [6, 2, 1, 5, 4, 3]]
[200, 218, 194, 194, 202, 203, 213, 181]
[50.0, 45.87155963302752, 51.54639175257732, 51.54639175257732, 49.504950495049506, 49.26108374384236, 46.948356807511736, 55.248618784530386]
[[4, 3, 6, 1, 5, 2], [6, 1, 2, 4, 5, 3], [6, 2, 1, 5, 4, 3], [1, 5, 4, 6, 3, 2], [1, 6, 3, 2, 4, 5], [2, 1, 4, 5, 6, 3]]
[3, 4]
[[6, 2, 1, 4, 5, 3], [6, 1, 2, 5, 4, 3], [1, 4, 5, 2, 6, 3], [1, 6, 3, 5, 2, 4], [4, 3, 1, 6, 5, 2], [5, 4, 6, 1, 3, 2]]
[0, 3]
(2, [2, 4, 5, 1, 6, 3])
49.58388205996657
51.33305112274803
[181, 194, 194, 194, 196, 199, 200, 202]
[6, 2, 1, 5, 4, 3]
177
1
[[6, 2, 1, 5, 4, 3], [2, 4, 5, 1, 6, 3], [1, 6, 3, 2, 4, 5], [6, 1, 2, 4, 5, 3], [4, 3, 1, 6, 5, 2], [6, 2, 1, 4, 5, 3], [4, 3, 6, 1, 5, 2], [1, 5, 4, 6, 3, 2]]
[181, 194, 194, 194, 196, 199, 200, 202]
[55.248618784530386, 51.54639175257732, 51.54639175257732, 51.5463917